In [ ]:
!pip install scholarly

In [3]:
import json
from scholarly import scholarly
from tqdm.notebook import tqdm

# Co-Authors Network

In [89]:
nodes = []
edges = []
nodeSet = set()

In [90]:
first_author_name = "Edward Elson Kosasih"
first_author = next(scholarly.search_author(first_author_name))
first_author = scholarly.fill(first_author)
first_author

nodes.append({"id": first_author["name"], "group": 0})
nodeSet.add(first_author["name"])

In [82]:
# get tier 1 coauthors
tier1_coauthors = first_author["coauthors"]
tier1_coauthors

for tier1_coauthor in tier1_coauthors:
  # add nodes
  if tier1_coauthor["name"] not in nodeSet:
    nodes.append({"id": tier1_coauthor["name"], "group": 1})
    nodeSet.add(tier1_coauthor["name"])

  # add edges
  edges.append({"source": first_author["name"], "target": tier1_coauthor["name"], "value": 0})

In [83]:
# tier 2 coauthors
for tier1_coauthor in tier1_coauthors:
  tier1_coauthor_details = scholarly.fill(scholarly.search_author_id(tier1_coauthor["scholar_id"]))
  tier2_coauthors_i = tier1_coauthor_details["coauthors"]

  for tier2_coauthor_i in tier2_coauthors_i:
    # skip first author
    if tier2_coauthor_i["name"] != first_author_name:
      # add nodes
      if tier2_coauthor_i["name"] not in nodeSet:
        nodes.append({"id": tier2_coauthor_i["name"], "group": 2})
        nodeSet.add(tier2_coauthor_i["name"])

      # add edges
      edges.append({"source": tier1_coauthor["name"], "target": tier2_coauthor_i["name"], "value": 1})

In [86]:
with open('coauthor_citations.json', 'w') as f:
  json.dump({"nodes": nodes, "links": edges}, f, indent=4)

# Publications

In [4]:
first_author_name = "Edward Elson Kosasih"
first_author = next(scholarly.search_author(first_author_name))
first_author = scholarly.fill(first_author)
# first_author

In [ ]:
nodes = []
edges = []
nodeSet = set()

# central node is myself
nodes.append({"id": first_author_name, "group": 0})
nodeSet.add(first_author_name)

# add publications
for _pub in first_author["publications"]:
  # search for details
  pub = next(scholarly.search_pubs(_pub["bib"]["title"]))
  name = "{}. {}. {}. {}".format(", ".join(pub["bib"]["author"]), pub["bib"]["pub_year"], pub["bib"]["title"], pub["bib"]["venue"])

  # add my publication node
  nodes.append({"id": name, "group": 1})
  nodeSet.add(name)

  # add edges
  edges.append({"source": first_author_name, "target": name, "value": 0})

  # add publications citing them
  for cite in tqdm(scholarly.citedby(_pub)):
    citename = "{}. {}. {}. {}".format(", ".join(cite["bib"]["author"]), cite["bib"]["pub_year"], cite["bib"]["title"], cite["bib"]["venue"])
  
    if citename not in nodeSet:
      nodes.append({"id": citename, "group": 2})
      nodeSet.add(citename)

    # add edges
    edges.append({"source": name, "target": citename, "value": 1})


In [24]:
with open('publication_citations.json', 'w') as f:
  json.dump({"nodes": nodes, "links": edges}, f, indent=4)